In [11]:
import pandas as pd
from functions_load_and_transform import schedule, player_list, scoringDF

from datetime import datetime


In [12]:
def calc_score(row): # check if player guessed correctly
    if row[player] == row["Winner"]:
        return 1
    else:
        return 0

In [13]:
def check_winner(row): # winner column in game Data
    if row["Score Home"] > row["Score Guest"]:
        return row["Home Team"]
    else:
        return row["Away Team"]

In [14]:
schedule = pd.read_csv("data/schedule.csv", delimiter=";")
schedule["Date"] = pd.to_datetime(schedule["Date"], format="%Y.%m.%d %H:%M")
schedule["Score Home"] = 0
schedule["Score Guest"] = 0
schedule["Winner"] = schedule.apply(check_winner, axis=1)

def create_empty_bets_csv():
    betsDF = pd.DataFrame(columns=player_list)
    betsDF[["Week", "Game Nr."]] = schedule[["Week", "Game Nr."]]
    betsDF.to_csv("data/bets_2024.csv", index=False)

# create_empty_bets_csv()
# betsDF = pd.concat([schedule, betsDF])

In [6]:
def bets_input(week_nr, player, bets):
    df = pd.read_csv("data/bets_2024.csv")
    df.loc[df["Week"] == week_nr, player] = bets
    df.to_csv("data/bets_2024.csv", index=False)

In [7]:
week_nr = 1

with open(f"data/tipps_week_{week_nr}.txt") as f:
    for line in f.readlines():
        x = line.split("; ")
        y = x[1].replace("[", "").replace("'", "").replace("]\n", "")
        y = y.split(",")
        bets_input(week_nr=week_nr, player=x[0], bets=y)

df = pd.read_csv("data/bets_2024.csv")

df.head(30)

,Alex,Alina,Evelyn,Christopher,Ludwig,Manu,Natalie,Nikolai,Sebastian,Vero,Viki,Wolfgang,Week,Game Nr.
0,Kansas Chiefs,NaN,NaN,NaN,Kansas City Chiefs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,Eagles,NaN,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2
2,Atlanta Falcons,NaN,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,3
3,Buffalo Bills,NaN,NaN,NaN,Buffalo Bills,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4
4,Chicago Bears,NaN,NaN,NaN,Chicago Bears,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
5,Cincinnati Bengals,NaN,NaN,NaN,Cincinnati Bengals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6
6,Indianapolis Colts,NaN,NaN,NaN,Indianapolis Colts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,7
7,Miami Dolphins,NaN,NaN,NaN,Miami Dolphins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,8
8,New Orleans Saints,NaN,NaN,NaN,New Orleans Saints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,9
9,New York Giants,NaN,NaN,NaN,New York Giants,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,10


In [ ]:
scoringDF = pd.DataFrame(columns=player_list) # collect the points for each game
scoringDF["Week"] = schedule["Week"]

schedule


for player in player_list:
    scoringDF[player] = betsDF.apply(calc_score, axis=1)

In [15]:
schedule.loc[schedule["Week"] == 1, ["Home Team", "Away Team"]]

,Home Team,Away Team
0,Kansas City Chiefs,Baltimore Ravens
1,Philadelphia Eagles,Green Bay Packers
2,Atlanta Falcons,Pittsburgh Steelers
3,Buffalo Bills,Arizona Cardinals
4,Chicago Bears,Tennessee Titans
5,Cincinnati Bengals,New England Patriots
6,Indianapolis Colts,Houston Texans
7,Miami Dolphins,Jacksonville Jaguars
8,New Orleans Saints,Carolina Panthers
9,New York Giants,Minnesota Vikings


In [8]:
# daten laden
# Tipps eintragen
# Scoring berechnen
## bets und schedule combinieren
# bets.csv überschreiben

In [10]:
last_week = 1

filtered_DF = schedule.loc[schedule["Week"] == last_week, ["Home Team", "Away Team"]]

for h_team, a_team in zip(filtered_DF["Home Team"], filtered_DF["Away Team"]):
    print(h_team)
    print(a_team)

Kansas City Chiefs
Baltimore Ravens
Philadelphia Eagles
Green Bay Packers
Atlanta Falcons
Pittsburgh Steelers
Buffalo Bills
Arizona Cardinals
Chicago Bears
Tennessee Titans
Cincinnati Bengals
New England Patriots
Indianapolis Colts
Houston Texans
Miami Dolphins
Jacksonville Jaguars
New Orleans Saints
Carolina Panthers
New York Giants
Minnesota Vikings
Los Angeles Chargers
Las Vegas Raiders
Seattle Seahawks
Denver Broncos
Cleveland Browns
Dallas Cowboys
Tampa Bay Buccaneers
Washington Commanders
Detroit Lions
Los Angeles Rams
San Francisco 49ers
New York Jets
